# Yelp API Restaurant Calls By GPS Coordinates

For each coordinate listed in grid_coordinates2.csv, return first 1k restaurants found via yelp api and extract data regarding:

- name
- address & zip
- coordinates
- rating
- review count
- price level
- category
- yelp id

Write data to restaurant_data_coord.csv

In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv

# Import API key
from api_keys import api_key

### Perform API calls

- Dataframe from grid_coordinates.csv
- Build function that calls and writes restaurant data to new csv, given pair of coordinates
- Run list of coordinates through function

In [2]:
# Create dataframe from csv file
coords_df = pd.read_csv("Resources/grid_coordinates2.csv", 
                        names=["Lat", "Lng"])
coords_df

,Lat,Lng
0,38.842665,-123.072800
1,38.842665,-123.017222
2,38.842665,-122.961644
3,38.842665,-122.906066
4,38.842665,-122.850488
5,38.842665,-122.794910
6,38.842665,-122.739332
7,38.842665,-122.683754
8,38.842665,-122.628176
9,38.842665,-122.572598


In [4]:
# Function returns up to 1000 restaurant listings for input coordinates
def get_restaurants(lat, lng, api_key):
    
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization": "Bearer %s" % api_key}
    restaurant_data = []
    count = 0
    
    # Increases search return limit
    for offset in range(0, 1000, 50):
        
        # Set parameters and pass into API calls, radius 8046 meters = 5 miles
        params = {"term":"restaurants", "latitude":lat, "longitude":lng, "radius":3412, 
                  "limit":50, "offset":offset}
        req = requests.get(url, params=params, headers=headers)
        
        # Breaks if error occurs with search
        if req.status_code == 400:
            break
        elif req.status_code == 200:
            
            # Convert to json
            response = req.json()
            
            # Log history
            count += 1
            print(f"Now processing query set {count} of max 20")
            
            # Breaks if no further entries in query
            if response["businesses"] == []:
                break
            
            else:
                # Iterate through business results and extract data
                for biz in response["businesses"]:
                    
                    # Logic to replace missing price level data with NaN
                    if "price" not in biz:
                        
                        # Replace both price & missing category data with empty string
                        if biz["categories"] == []:
                            restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], 
                                                    biz["coordinates"]["latitude"], 
                                                    biz["coordinates"]["longitude"], 
                                                    biz["location"]["address1"], 
                                                    biz["location"]["zip_code"], biz["rating"], 
                                                    biz["review_count"], "", "", biz["id"]])
                        else:
                            restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], 
                                                    biz["coordinates"]["latitude"], 
                                                    biz["coordinates"]["longitude"], 
                                                    biz["location"]["address1"], 
                                                    biz["location"]["zip_code"], biz["rating"], 
                                                    biz["review_count"], "", biz["categories"][0]["title"], 
                                                    biz["id"]])

                    # Replace missing category data with empty string
                    elif biz["categories"] == []:
                        restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], "", biz["id"]])

                    else:
                        restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], 
                                                biz["categories"][0]["title"], biz["id"]])

    # Write to csv
    with open('Resources/restaurant_data_coords2.csv', 'a', encoding="utf-8") as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(restaurant_data)
    csvFile.close()
    
    # Returns total count of restaurants in coordinates
    return response["total"]


In [5]:
# Track # of coordinates processed
count = 0

# List to track total restaurants found in coordinate
totals_count = []

print("LOG HISTORY OF API CALLS:")
print("---------------------------")

# Loop thru list of coordinates
lat_coords = [lat for lat in coords_df["Lat"]]
lng_coords = [lng for lng in coords_df["Lng"]]

for index in range(len(lat_coords)):
                   
    #  Call get_restaurants fn and append to total_count list
    totals_count.append(get_restaurants(lat_coords[index], lng_coords[index] , api_key))
    
    # Print log history
    count += 1
    rem = len(lat_coords) - count
    print("-----------------------------------------")
    if count == len(lat_coords):
        print("Full list of coordinates processed!")
    elif count == len(lat_coords) - 1:
        print("Now getting results for final coordinates. Almost there!")
        print("-----------------------------------------")
    else:
        print(f"Data retrieval for coordinates complete")
        print(f"Getting results for next coordinates.. there are {rem} coordinates left")
        print("-----------------------------------------")

coords_df["Total # of Restaurants"] = totals_count

LOG HISTORY OF API CALLS:
---------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1304 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1303 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1302 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1301 coordinates left
-----------------------

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1270 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1269 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1268 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1267 coordinates left
----------------------------------------

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1236 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1235 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1234 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1233 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1201 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1200 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1199 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1198 coordinates left
---

Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1168 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1167 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1166 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 1165 coordinates left
---

...

Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 148 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 147 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 146 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. t

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 114 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 113 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 112 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 111 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
---------------------------------------

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 79 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 78 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 77 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 76 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
D

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 45 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 44 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 43 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 42 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
------

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 12 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 11 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 10 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 9 coordinates left
------------

In [6]:
# Show coords_df with updated total # of restaurants
coords_df.head()

,Lat,Lng,Total # of Restaurants
0,38.842665,-123.072800,0
1,38.842665,-123.017222,11
2,38.842665,-122.961644,0
3,38.842665,-122.906066,0
4,38.842665,-122.850488,0


In [7]:
# Save restaurants counts to csv
coords_df.to_csv("Resources/total_count_coords2.csv")